In [3]:
import ipywidgets as widgets
from IPython.display import display
import yaml
import numpy as np

In [35]:
fit_func_name1 = widgets.Dropdown(options = ["PWL", "LP", "MBPWL", "freeLP"], description = "Function to fit", style= {'description_width': 'initial'})
Telescope1 = widgets.Dropdown(options = ["MAGIC", "CTAN_alpha"], description = "Telescope", style= {'description_width': 'initial'})
Spectrum_func_name1 = widgets.Dropdown(options = ["PWL", "LP"], description = "Simulated spectrum function", style= {'description_width': 'initial'})
EBL_Model1 = widgets.Dropdown(options = ["Dominguez"], description = "EBL model", style= {'description_width': 'initial'})
Source_flux1 = widgets.FloatText(value=8.7e-6, min=1e-8, max=1e-3, step=1e-7, description='Source flux:', disabled=False, continuous_update=True, style= {'description_width': 'initial'})
Observation_time1 = widgets.IntSlider(value = 42480, min = 1e3, max = 1e5, step= 1e3, description = "Observation time", indent = True, style= {'description_width': 'initial'})

niter1 = widgets.IntSlider(value = 1000, min = 1, max = 1e5, step= 10, description = "Number of simulations", style= {'description_width': 'initial'})
Energy_migration1 = widgets.Checkbox(value=True, description='Energy migration', indent=True, style= {'description_width': 'initial'})
Background1 = widgets.Checkbox(value=True, description='Simulated Bacgkround', indent=True, style= {'description_width': 'initial'})
Background_scale1 = widgets.FloatSlider(value = 1., min = 0., max = 3, step= 1e-2, description = "Background scale", style= {'description_width': 'initial'})
Forward_folding1 = widgets.Checkbox(value=True, description='Forward folding', indent=True, style= {'description_width': 'initial'})
box2 = widgets.HBox([Forward_folding1, Energy_migration1])
box3 = widgets.HBox([Background1, Background_scale1])
first_bin1 = widgets.FloatSlider(min = 0., max = 1., value = 0., step = 0.01, description = "First alpha bin", style= {'description_width': 'initial'})
last_bin1 = widgets.FloatSlider(min = 1., max = 2., value = 2., step = 0.01, description = "Last alpha bin", style= {'description_width': 'initial'})
step1  = widgets.FloatSlider(min = 0.01, max = 0.1, value = 0.05, step = 0.01, description = "Alpha bin width (step)", style= {'description_width': 'initial'})
initial_guess_pos1 = widgets.FloatSlider(value = 1.2, description = "Initial guess alpha position", style= {'description_width': 'initial'})
widgets.jslink((first_bin1, 'value'), (initial_guess_pos1, 'min'))
widgets.jslink((last_bin1, 'value'), (initial_guess_pos1, 'max'))
widgets.jslink((step1, 'value'), (initial_guess_pos1, 'step'))
widgets.jslink((step1, 'value'), (last_bin1, 'step'))
widgets.jslink((step1, 'value'), (first_bin1, 'step'))
box4 = widgets.HBox([first_bin1,last_bin1,step1, initial_guess_pos1])

#This has to be changed when more Simulated functions
Norm1 = widgets.FloatSlider(min = 0.1, max = 10, value = 0.25, step = 0.1, disabled=True, description="Normalization Energy in TeV", style= {'description_width': 'initial'})
Ph_index1 = widgets.FloatSlider(value =2.03, min = 1., max = 3., step = 0.01, description = "Photon index", style= {'description_width': 'initial'})
box1 = widgets.HBox([Source_flux1,Observation_time1, Norm1, Ph_index1])
# box_layout = widgets.Layout(display='flex',
#                 flex_flow='column',
#                 align_items='center',
#                 width='50%')
box = widgets.VBox(children=[Telescope1, EBL_Model1, niter1, Spectrum_func_name1, box1, fit_func_name1, box2, box3, box4])
display(box)

In [36]:
fit_func_name = fit_func_name1.value
Telescope = Telescope1.value
Spectrum_func_name = Spectrum_func_name1.value 
EBL_Model = EBL_Model1.value 
Source_flux = Source_flux1.value 
Observation_time = Observation_time1.value
niter = niter1.value
Energy_migration = Energy_migration1.value
Background = Background1.value
Background_scale = Background_scale1.value
Forward_folding = Forward_folding1.value
initial_guess_pos = initial_guess_pos1.value
step = step1.value
last_bin = last_bin1.value
first_bin = first_bin1.value
Norm = Norm1.value
Ph_index = Ph_index1.value

In [40]:
if fit_func_name == "MBPWL":
    knots1 = widgets.IntSlider(value = 3, min = 3, max = 20, description="Number of knots", style= {'description_width': 'initial'}) #Number of knots of the function. 3 is the default.
    Efirst1 = widgets.FloatSlider(value = 0.2, min = 0.01, max = 1., step = 0.01, description = "Energy of the first knot", style= {'description_width': 'initial'}) #Energy of the first knot
    Elast1 = widgets.FloatSlider(value = 1.1, min = 1., max = 4., step = 0.1, description = "Energy of the last knot", style= {'description_width': 'initial'})
    #the rest of the bins are distributed logaritmically between those two values
    display(widgets.HBox([knots1, Efirst1, Elast1]))
#for PWL
elif fit_func_name == "PWL":
    if Forward_folding:
        initial_guess_0 = np.array([5e-6, 2.00])
    else:
        initial_guess_0 = np.array([5e-7, 2.]) #phi_0, gamma
#for LP and freeLP
elif fit_func_name == "LP" or fit_func_name == "freeLP":
    if Forward_folding:
        initial_guess_0 = np.array([5e-6, 2., 0.])
    else:
        initial_guess_0 = np.array([5e-7, 2., 0.]) #phi_0, alpha, beta

LP_curvature = widgets.FloatSlider(value = 0.01, min = -1, max = 1, step = 0.005, description="Log-Parabola curvature", style= {'description_width': 'initial'}, readout_format='.3f')
display(widgets.HBox([LP_curvature]))


In [41]:
if fit_func_name == "MBPWL":
    knots = knots1.value
    if Forward_folding:
        initial_guess_0 = np.zeros(knots+2)
        initial_guess_0[0] = 5e-6
        initial_guess_0[1] = 2.
    else:
        initial_guess_0 = np.zeros(knots+2)
        initial_guess_0[0] = 5e-6
        initial_guess_0[1] = 2.

    config = dict(
        fit_func_name = fit_func_name1.value,
        Telescope = Telescope1.value,
        Spectrum_func_name = Spectrum_func_name1.value, 
        EBL_Model = EBL_Model1.value, 
        Source_flux = Source_flux1.value, 
        Observation_time = Observation_time1.value,
        niter = niter1.value,
        Energy_migration = Energy_migration1.value,
        Background = Background1.value,
        Background_scale = Background_scale1.value,
        Forward_folding = Forward_folding1.value,
        initial_guess_pos = initial_guess_pos1.value,
        step = step1.value,
        last_bin = last_bin1.value,
        first_bin = first_bin1.value,
        Norm = Norm1.value,
        Ph_index = Ph_index1.value,
        knots = knots1.value,
        Efirst = Efirst1.value,
        Elast = Elast1.value,
        initial_guess_0 = initial_guess_0.tolist(),
        LP_curvature = LP_curvature.value,
    )
else:    
    config = dict(
        fit_func_name = fit_func_name1.value,
        Telescope = Telescope1.value,
        Spectrum_func_name = Spectrum_func_name1.value, 
        EBL_Model = EBL_Model1.value, 
        Source_flux = Source_flux1.value, 
        Observation_time = Observation_time1.value,
        niter = niter1.value,
        Energy_migration = Energy_migration1.value,
        Background = Background1.value,
        Background_scale = Background_scale1.value,
        Forward_folding = Forward_folding1.value,
        initial_guess_pos = initial_guess_pos1.value,
        step = step1.value,
        last_bin = last_bin1.value,
        first_bin = first_bin1.value,
        Norm = Norm1.value,
        Ph_index = Ph_index1.value,
        initial_guess_0 = initial_guess_0.tolist(),
        LP_curvature = LP_curvature.value,
    )

with open('EBL_fit_config.yml', 'w') as outfile:
    #outfile.write("# Input data for MBPWLopt.py \n")
    yaml.dump(config, outfile, default_flow_style=False)

In [42]:
with open("EBL_fit_config.yml", "r") as f:
    inp_config = yaml.safe_load(f)

print(inp_config)

{'Background': True, 'Background_scale': 1.0, 'EBL_Model': 'Dominguez', 'Efirst': 0.2, 'Elast': 1.1, 'Energy_migration': True, 'Forward_folding': True, 'LP_curvature': 0.01, 'Norm': 0.25, 'Observation_time': 42480, 'Ph_index': 2.03, 'Source_flux': 8.7e-06, 'Spectrum_func_name': 'PWL', 'Telescope': 'MAGIC', 'first_bin': 0.0, 'fit_func_name': 'MBPWL', 'initial_guess_0': [5e-06, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'initial_guess_pos': 1.2, 'knots': 5, 'last_bin': 2.0, 'niter': 1000, 'step': 0.05}
